# 2. 텍스트 데이터 준비

### `TextVectorization` 층 사용하기

In [ ]:
import string

class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [ ]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
		## 정수 인덱스로 인코딩된 단어 시퀀스를 반환하도록 층을 설정
		## 여러가지 다른 출력 모드도 있음
		## 벡터화된 텍스트는 정수 시퀀스로 표현
    output_mode="int",
)

In [5]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

In [6]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

**어휘 사전 출력하기**

In [7]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [8]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [9]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


# 3. 단어 그룹을 표현하는 두 가지 방법: 집합과 시퀀스

### IMDB 영화 리뷰 데이터 준비하기

In [10]:
%curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
%tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  29.6M      0  0:00:02  0:00:02 --:--:-- 29.6M


In [12]:
%cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [13]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    ## 코드를 여러 번 실행해도 동일한 검증 세트가 만들어지도록 랜덤 시드를 지정하여 훈련 파일 목록을 섞음
    random.Random(1337).shuffle(files)
    ## 훈련 파일 중 20%를 검증 세트로 덜어 냄
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    ## 파일을 aclImdb/val/neg와 aclImdb/val/pos로 옮김
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

In [14]:
from tensorflow import keras
batch_size = 32

## 이 라인을 실행하면 "Found 20000 files belonging to 2 classes."가 출력될 것
## 다른 문장이 출력되면 aclImdb/train/unsup 디렉터리 삭제 후 다시 실행
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**첫 번째 배치의 크기와 dtype 출력하기**

In [15]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"After you see Vertigo, then watch Bell, Book and Candle, made within months of each other.<br /><br />My second favorite Kim Novak film, with Picnic, coming in as third.<br /><br />All three performances are great, Vertigo, being the best, of all.<br /><br />They came to my nowhere Kansas Prairie town, near by, at Salina, Kansas in the 50s, to film, Picnic. <br /><br />Bell, Book and Candle's musical score, I believe is one of Alex North's. Perfect for this bit of comedy.<br /><br />After Vertigo, Stewart and Novak, did this comedy, how amusing to note the dramatic contrast.<br /><br />Worth your time, if you like Kim Novak. The Greta Garbo of my youth.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


### 단어를 집합으로 처리하기: BoW 방식

#### Single words (unigrams) with binary encoding

**`TextVectorization` 층으로 데이터 전처리하기**

In [16]:
text_vectorization = TextVectorization(
    ## 가장 많이 등장하는 2만 개의 단어로 어휘 사전을 제한
    ## 그렇지 않으면 훈련 데이터에 있는 모든 단어를 인덱싱하게 되어 비 효율적
    ## 일반적으로 텍스트 분류에서 2만 개는 적절한 어휘 사전 크기
    max_tokens=20000,
    output_mode="multi_hot", ## 멀티-핫 이진 벡터로 출력 토큰을 인코딩
)
## (레이블 없이) 원시 텍스트 입력만 반환하는 데이터셋을 준비
## text_only_train_ds: text, label 쌍
## x(text)를 반환
text_only_train_ds = train_ds.map(lambda x, y: x) 
text_vectorization.adapt(text_only_train_ds) ## adapt()로 이 데이터셋의 어휘 사전을 인덱싱 -> 효울작인 텍스트-숫자 변환을 준비

## 훈련, 검증, 테스트 데이터셋을 전처리
## 다중 CPU코어를 활용하기 위해 num_parallel_calls 매개변수를 지정
## text_vectorization(x): 멀티-핫 인코딩으로 변환
## (text_vectorization(x), y): 텍스트 데이터가 변환되지만 레이블은 동일하게 유지되는 튜플을 반환
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**이진 유니그램 데이터셋의 출력 확인하기**

In [17]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


**모델 생성 유틸리티**

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

**이진 유니그램 모델 훈련하고 테스트하기**

In [21]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.h5",
                                    save_best_only=True)
]
## 데이터셋의 cache() 메서드를 호출하여 메모리에 캐싱
## 이렇게 하면 첫 번째 에포크에서 한 번만 전처리하고 이후 에포크에서는 전처리도니 텍스트를 재사용
## 메모리에 들어갈 만큼 작은 데이터일 때 사용할 수 있음
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.h5")
print(f"테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 11s 14ms/step - loss: 0.4008 - accuracy: 0.8306 - val_loss: 0.2689 - val_accuracy: 0.8946
Epoch 2/10


#### 이진 인코딩을 사용한 바이그램

**바이그램을 반환하는 `TextVectorization` 층 만들기**

In [22]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

**이진 바이그램 모델 훈련하고 테스트하기**

In [23]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.h5",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.h5")
print(f"테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_6 (Dense)             (None, 16)                320016    
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.3901 - accuracy: 0.8371 - val_loss: 0.2620 - val_accuracy: 0.8972
Epoch 2/10
6

#### TF-IDF 인코딩을 사용한 바이그램

**토큰 카운트를 반환하는 `TextVectorization` 층**

In [24]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

**TF-IDF 가중치가 적용된 출력을 반환하는 `TextVectorization` 층**

In [25]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)

**TF-IDF 바이그램 모델 훈련하고 테스트하기**

In [26]:

## adapt()를 호출하면 어휘 사전과 TF-IDF 가중치를 학습
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.h5", save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.h5")
print(f"테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_8 (Dense)             (None, 16)                320016    
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 6s 9ms/step - loss: 0.5076 - accuracy: 0.7673 - val_loss: 0.2983 - val_accuracy: 0.8894
Epoch 2/10
62

In [27]:
inputs = keras.Input(shape=(1,), dtype="string") ## 하나의 입력 샘플은 하나의 문자열
processed_inputs = text_vectorization(inputs) ## 텍스트 전처리 수행: 입력된 텍스트 데이터를 2-garm 벡터로 변환
outputs = model(processed_inputs) ## 전처리된 텍스트 벡터를 model에 입력하여 예측을 수행
inference_model = keras.Model(inputs, outputs) ## 예측 결과를 반환 (end-to-end)

In [28]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"긍정적인 리뷰일 확률: {float(predictions[0] * 100):.2f} 퍼센트")

긍정적인 리뷰일 확률: 93.58 퍼센트


#### 첫 번째 예제

**데이터 다운로드**

In [ ]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4916k      0  0:00:16  0:00:16 --:--:-- 8263k


**데이터 준비**

In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**정수 시퀀스 데이터셋 준비하기**

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    ## 적당한 입력 크기를 유지하기 위해 입력에서 600개 단어 이후는 잘라 버림
    ## 평균 리뷰 길이가 233개의 단어고 600개의 단어보다 긴 리뷰는 5%뿐이므로 합리적인 선택
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델 만들기**

In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64") ## 입력은 정수 시퀀스
embedded = tf.one_hot(inputs, depth=max_tokens) ## 정수를 20,000차원의 이진 벡터로 인코딩
x = layers.Bidirectional(layers.LSTM(32))(embedded) ## 양방향 LSTM 층을 추가
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x) ## 층을 분류
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

**첫 번째 시퀀스 모델 훈련하기**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 60s 96ms/step - loss: 0.4053 - accuracy: 0.8417 - val_loss: 0.3337 - val_accuracy: 0.8776
Epoch 2/10
  1/625 [..............................] - ETA: 58s - loss: 0.3699 - accuracy: 0.8750

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 60s 95ms/step - loss: 0.3114 - accuracy: 0.8871 - val_loss: 0.3029 - val_accuracy: 0.8908
Epoch 3/10
625/625 [==============================] - 60s 95ms/step - loss: 0.2707 - accuracy: 0.9071 - val_loss: 0.3504 - val_accuracy: 0.8510
Epoch 4/10
625/625 [==============================] - 60s 95ms/step - loss: 0.2336 - accuracy: 0.9218 - val_loss: 0.3676 - val_accuracy: 0.8836
Epoch 5/10
625/625 [==============================] - 60s 95ms/step - loss: 0.2042 - accuracy: 0.9313 - val_loss: 0.2786 - val_accuracy: 0.8920
Epoch 6/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1811 - accuracy: 0.9420 - val_loss: 0.3031 - val_accuracy: 0.8924
Epoch 7/10
625/625 [==============================] - 60s 95ms/step - loss: 0.1615 - accuracy: 0.9481 - val_loss: 0.3695 - val_accuracy: 0.8890
Epoch 8/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1442 - accuracy: 0.9547 - val_loss: 0.3390 - val_accuracy: 0.8924
Epo

#### 단어 임베딩 이해하기

#### 임베딩 층으로 단어 임베딩 학습하기

**`Embedding` 층 만들기**

In [ ]:
## Embedding 층은 적어도 2개의 매개변수(가능한 토큰의 개수, 임베딩 차원)가 필요함
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**밑바닥부터 훈련하는 `Embedding` 층을 사용한 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

#### 패딩과 마스킹 이해하기

**마스킹을 활성화한 `Embedding` 층 사용하기**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_with_masking.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

#### 사전 훈련된 단어 임베딩 사용하기

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-11-04 03:32:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-11-04 03:32:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-11-04 03:32:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**GloVe 단어 임베딩 파일 파싱하기**

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {} ## 딕셔너리
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1) ## 각줄을 단어(word)와 벡터(coefs)로 나늠
        coefs = np.fromstring(coefs, "f", sep=" ") ## 벡터를 실수 배열로 뱐환
        embeddings_index[word] = coefs ## 변환된 벡터를 embeddings_index 딕셔너리에 단어를 키로 하여 저장

print(f"단어 벡터 개수: {len(embeddings_index)}") ## 단어 벡터 개수: 400000

단어 벡터 개수: 400000


**GloVe 단어 임베딩 행렬 준비하기**

In [ ]:
embedding_dim = 100

## TextVectorization 층에 인덱싱된 단어를 추출
vocabulary = text_vectorization.get_vocabulary()
## 어휘 사전에 있는 단어와 인덱스를 매핑
word_index = dict(zip(vocabulary, range(len(vocabulary))))

## GloVe 벡터로 채울 행렬을 준비
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens: ## word의 index가 20,000(max_tokens or num_words)보다 작은 경우만
        embedding_vector = embeddings_index.get(word) ## word에 해당하는 임베딩 벡터
    ## 인덱스 i에 대한 단어 벡터로 행렬의 i번째 항목을 채움
    ## 임베딩 인덱스에 없는 단어는 모두 0이 됨
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    ## 사전 학습된 임베딩으로 초기화
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False, ## 사전 학습된 임베딩 벡터를 그대로 사용
    mask_zero=True, ## 패딩 처리: 0으로 마스킹된 값을 무시
)

**사전 훈련된 임베딩을 사용하는 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.h5")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________

# 4. 트랜스포머 아키텍처

**데이터 가져오기**

In [ ]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  21.3M      0  0:00:03  0:00:03 --:--:-- 21.3M


**데이터 준비**

In [ ]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**데이터 벡터화**

In [ ]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**`Layer` 층을 상속하여 구현한 트랜스포머 인코더**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs) ## 상위 클래스(layers.Layer) 초기화
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    ## call()에서 연산을 수행
    def call(self, inputs, mask=None):
        ## Embedding 층에서 생성하는 마스크는 2D이지만 어텐션 층은 3D or 4D를 기대하므로 랭크를 늘림
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    ## 모델을 저장할 수 있도록 직렬화를 구현
    ## JSON 문자열 또는 바이트 스트림과 같이 쉽게 저장하거나 전송할 수 있는 형식으로 변환하는 프로세스
    def get_config(self):
        ## 상위 슈퍼클래스의 Layer 클래스
        ## TransformerEncoder 인스턴스에서 get_config()를 호출하면
        ## 모델 저장 및 로드와 같은 직렬화 목적에 사용, 객체를 저장, 전송 및 재구성
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더를 사용하여 텍스트 분류하기**

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) ## (None, None, embed_dim)
## TransformerEncoder는 전체 시퀀스를 반환하기 때문에
## 분류작업을 위해 전역 풀링 층으로 각 시퀀스를 하나의 벡터로 만듦 (None, embed_dim)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x) ## (None, 1)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

**트랜스포머 인코더 기반 모델 훈련하고 평가하기**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.h5",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20
625/625 [==============================] - 49s 78ms/step - loss: 0.3364 - accuracy: 0.8550 - val_loss: 0.3118 - val_accuracy: 0.8732


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/20
625/625 [==============================] - 43s 69ms/step - loss: 0.3045 - accuracy: 0.8699 - val_loss: 0.2991 - val_accuracy: 0.8732
Epoch 3/20
625/625 [==============================] - 44s 70ms/step - loss: 0.2724 - accuracy: 0.8887 - val_loss: 0.2877 - val_accuracy: 0.8802
Epoch 4/20
625/625 [==============================] - 44s 70ms/step - loss: 0.2435 - accuracy: 0.9021 - val_loss: 0.2880 - val_accuracy: 0.8804
Epoch 5/20
625/625 [==============================] - 43s 69ms/step - loss: 0.2141 - accuracy: 0.9158 - val_loss: 0.2930 - val_accuracy: 0.8814
Epoch 6/20
625/625 [==============================] - 42s 66ms/step - loss: 0.1861 - accuracy: 0.9284 - val_loss: 0.3078 - val_accuracy: 0.8786
Epoch 7/20
625/625 [==============================] - 43s 68ms/step - loss: 0.1538 - accuracy: 0.9419 - val_loss: 0.3307 - val_accuracy: 0.8772
Epoch 8/20
625/625 [==============================] - 43s 69ms/step - loss: 0.1293 - accuracy: 0.9524 - val_loss: 0.3357 - val_accuracy:

#### 위치 인코딩을 사용해 위치 정보 주입하기

**서브클래싱으로 위치 임베딩 구현하기**

In [ ]:
class PositionalEmbedding(layers.Layer):
    ## 위치 임베딩의 단점: 시퀀스 길이를 미리 알아야 함
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        ## 토큰 인덱스를 위한 Embedding 층을 준비
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        ## 토큰 위치를 위한 ## 토큰 위치를 위한 Embedding 층을 준비 층을 준비
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim) 
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1] ## 600개의 Sqeuence 1
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions ## 두 임베딩 벡터를 더함

    ## Embedding 층 처럼 이 층은 입력에 있는 0 패팅을 무시할 수 있도록 마스킹을 생성해야 함
    ## compute_mask()는 프레임워크에 의해 자동으로 호출되고 만들어진 마스킹은 다음 층으로 전달됨
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### 텍스트 분류 트랜스포머

**트랜스포머 인코더와 위치 임베딩 합치기**

In [ ]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.h5",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           